### 级联操作
- pd.concat, pd.append

pandas使用pd.concat函数，与np.concatenate函数类似，只是多了一些参数：
```
objs
axis=0
keys
join='outer' / 'inner':表示的是级联的方式，outer会将所有的项进行级联（忽略匹配和不匹配），而inner只会将匹配的项级联到一起，不匹配的不级联
ignore_index=False
```

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame

- 匹配级联

In [2]:
df1 = DataFrame(data=np.random.randint(0,100,size=(5,3)),columns=['A','B','C'])
df2 = DataFrame(data=np.random.randint(0,100,size=(5,3)),columns=['A','D','C'])

In [3]:
df1

,A,B,C
0,43,9,40
1,87,33,30
2,83,81,78
3,95,17,98
4,75,90,92


In [4]:
df2

,A,D,C
0,39,21,11
1,6,74,16
2,57,26,29
3,86,78,15
4,95,90,58


In [6]:
pd.concat((df1,df1),axis=1)

,A,B,C,A,B,C
0,43,9,40,43,9,40
1,87,33,30,87,33,30
2,83,81,78,83,81,78
3,95,17,98,95,17,98
4,75,90,92,75,90,92


- 不匹配级联
    - 不匹配指的是级联的维度的索引不一致。例如纵向级联时列索引不一致，横向级联时行索引不一致
    - 有2种连接方式：
        - 外连接：补NaN（默认模式）
        - 内连接：只连接匹配的项

In [10]:
pd.concat((df1,df2),axis=0)

/Users/bobo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,A,B,C,D
0,43,9.0,40,NaN
1,87,33.0,30,NaN
2,83,81.0,78,NaN
3,95,17.0,98,NaN
4,75,90.0,92,NaN
0,39,NaN,11,21.0
1,6,NaN,16,74.0
2,57,NaN,29,26.0
3,86,NaN,15,78.0
4,95,NaN,58,90.0


In [12]:
pd.concat((df1,df2),axis=0,join='outer')

/Users/bobo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,A,B,C,D
0,43,9.0,40,NaN
1,87,33.0,30,NaN
2,83,81.0,78,NaN
3,95,17.0,98,NaN
4,75,90.0,92,NaN
0,39,NaN,11,21.0
1,6,NaN,16,74.0
2,57,NaN,29,26.0
3,86,NaN,15,78.0
4,95,NaN,58,90.0


- 如果想要保留数据的完整性必须使用outer（外连接）

### 合并操作
- merge与concat的区别在于，merge需要依据某一共同列来进行合并

- 使用pd.merge()合并时，会自动根据两者相同column名称的那一列，作为key来进行合并。

- 注意每一列元素的顺序不要求一致

#### 一对一合并

In [13]:
df1 = DataFrame({'employee':['Bob','Jake','Lisa'],
                'group':['Accounting','Engineering','Engineering'],
                })
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering


In [14]:
df2 = DataFrame({'employee':['Lisa','Bob','Jake'],
                'hire_date':[2004,2008,2012],
                })
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012


In [16]:
pd.merge(left=df1,right=df2,on='employee')

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004


#### 一对多合并

In [17]:
df3 = DataFrame({
    'employee':['Lisa','Jake'],
    'group':['Accounting','Engineering'],
    'hire_date':[2004,2016]})
df3

,employee,group,hire_date
0,Lisa,Accounting,2004
1,Jake,Engineering,2016


In [18]:
df4 = DataFrame({'group':['Accounting','Engineering','Engineering'],
                       'supervisor':['Carly','Guido','Steve']
                })
df4

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,Engineering,Steve


In [19]:
pd.merge(left=df3,right=df4,on='group')

,employee,group,hire_date,supervisor
0,Lisa,Accounting,2004,Carly
1,Jake,Engineering,2016,Guido
2,Jake,Engineering,2016,Steve


#### 多对多合并

In [20]:
df1 = DataFrame({'employee':['Bob','Jake','Lisa'],
                 'group':['Accounting','Engineering','Engineering']})
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering


In [21]:
df5 = DataFrame({'group':['Engineering','Engineering','HR'],
                'supervisor':['Carly','Guido','Steve']
                })
df5

,group,supervisor
0,Engineering,Carly
1,Engineering,Guido
2,HR,Steve


In [26]:
pd.merge(left=df1,right=df5,on='group',how='right')

,employee,group,supervisor
0,Jake,Engineering,Carly
1,Lisa,Engineering,Carly
2,Jake,Engineering,Guido
3,Lisa,Engineering,Guido
4,NaN,HR,Steve


#### key的规范化
- 当列冲突时，即有多个列名称相同时，需要使用on=来指定哪一个列作为key，配合suffixes指定冲突列名

In [27]:
df1 = DataFrame({'employee':['Jack',"Summer","Steve"],
                 'group':['Accounting','Finance','Marketing']})
df1

,employee,group
0,Jack,Accounting
1,Summer,Finance
2,Steve,Marketing


In [28]:
df2 = DataFrame({'employee':['Jack','Bob',"Jake"],
                 'hire_date':[2003,2009,2012],
                'group':['Accounting','sell','ceo']})
df2

,employee,hire_date,group
0,Jack,2003,Accounting
1,Bob,2009,sell
2,Jake,2012,ceo


In [29]:
pd.merge(df1,df2,on='group')

,employee_x,group,employee_y,hire_date
0,Jack,Accounting,Jack,2003


- 当两张表没有可进行连接的列时，可使用left_on和right_on手动指定merge中左右两边的哪一列列作为连接的列

In [30]:
df1 = DataFrame({'employee':['Bobs','Linda','Bill'],
                'group':['Accounting','Product','Marketing'],
               'hire_date':[1998,2017,2018]})
df1

,employee,group,hire_date
0,Bobs,Accounting,1998
1,Linda,Product,2017
2,Bill,Marketing,2018


In [31]:
df5 = DataFrame({'name':['Lisa','Bobs','Bill'],
                'hire_dates':[1998,2016,2007]})
df5

,name,hire_dates
0,Lisa,1998
1,Bobs,2016
2,Bill,2007


In [34]:
pd.merge(df1,df5,left_on='employee',right_on='name')

,employee,group,hire_date,name,hire_dates
0,Bobs,Accounting,1998,Bobs,2016
1,Bill,Marketing,2018,Bill,2007


#### 内合并与外合并:out取并集 inner取交集

In [ ]:
df6 = DataFrame({'name':['Peter','Paul','Mary'],
               'food':['fish','beans','bread']}
               )
df7 = DataFrame({'name':['Mary','Joseph'],
                'drink':['wine','beer']})


In [ ]:
df6 = DataFrame({'name':['Peter','Paul','Mary'],
               'food':['fish','beans','bread']}
               )
df7 = DataFrame({'name':['Mary','Joseph'],
                'drink':['wine','beer']})
